In [1]:
import sys 
sys.path.append('../../')

service_account_path = "../../creds/google__sa.json"
tracking_sheet_id = "1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4"

In [2]:
import pandas as pd

from src.sheets_utils import download_sheet_as_df


contributors_df = download_sheet_as_df(
    service_account_path,
    tracking_sheet_id,
    "Contributors"
)

tasks_df = pd.concat(
    [
        download_sheet_as_df(
            service_account_path,
            tracking_sheet_id,
            "Conversations_Batch_2"
        ),
        download_sheet_as_df(
            service_account_path,
            tracking_sheet_id,
            "Conversations_Batch_3"
        ),
        download_sheet_as_df(
            service_account_path,
            tracking_sheet_id,
            "Conversations_Batch_4"
        ),
        download_sheet_as_df(
            service_account_path,
            tracking_sheet_id,
            "Conversations_Batch_5"
        ),
        download_sheet_as_df(
            service_account_path,
            tracking_sheet_id,
            "Conversations_Batch_6"
        ),
    ],
    ignore_index=True
)
tasks_df = tasks_df[tasks_df["completion_status"] == "Done"].reset_index()


reviews_df = download_sheet_as_df(
    service_account_path,
    tracking_sheet_id,
    "Reviews"
)

In [16]:
# Parse Conversations into list of dicts

import io
import threading

import nbformat
from fuzzywuzzy import fuzz

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload


def get_closest_match(query, choices):
    """
    Get the closest match(es) to a query string from a list of choices.

    :param query: The query string.
    :param choices: A list of strings to match against.
    :param limit: The maximum number of matches to return.
    """
    best_role = None
    best_score = 0
    for choice in choices:
        score = fuzz.ratio(query, choice)
        if score > best_score and score > 25:
            best_score = score
            best_role = choice

    return best_role, best_score


def notebook_parser(notebook):
    """
    Parse a notebook and extract the message objects.

    :param notebook: The notebook object.
    """
    messages = []
    for cell in notebook.cells[2:]:
        if cell["cell_type"] == "markdown":
            markdown_headers = ["**User**", "**Assistant**"]
            lines = cell["source"].split("\n")
            first_line = lines[0]
            role, score = get_closest_match(first_line, markdown_headers)
            if score>25:
                message = {
                    "role": role.replace("*", "").strip(),
                    "content": "\n".join(lines[1:]).strip("\n"),
                    "type": "markdown"
                }
                messages.append(message)

        elif cell["cell_type"] == "code":
            code_headers = ["# User", "# Assistant"]
            lines = cell["source"].split("\n")
            first_line = lines[0]
            role, score = get_closest_match(first_line, code_headers)
            if score>25:
                message = {
                    "role": role.replace("#", "").strip(),
                    "content": "\n".join(lines[1:]).strip("\n"),
                    "type": "code"
                }
                messages.append(message)
    return messages


def download_and_parse_notebook(service_account_file, file_id):
    # Authenticate with the service account
    credentials = service_account.Credentials.from_service_account_file(
        service_account_file, scopes=['https://www.googleapis.com/auth/drive'])
    service = build('drive', 'v3', credentials=credentials)

    # Request to download the file
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    # Download the file
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print("Download progress: %d%%." % int(status.progress() * 100))

    # Move the buffer's pointer to the beginning
    fh.seek(0)

    # Open the notebook
    notebook = nbformat.read(fh, as_version=4)

    # Parse the notebook
    messages = notebook_parser(notebook)

    # # Extract the first cell
    first_cell = notebook.cells[0]
    lines = first_cell["source"].split("\n")
    metadata = {}
    for line in lines:
        if "**Python Topics**" in line:
            metadata["topic"] = line.split(" - ")[1]
        if "**Type**" in line:
            metadata["type"] = line.split(" - ")[1]
        if "**Target Number of Turns (User + Assistant)**" in line:
            metadata["target_turns"] = line.split(" - ")[1]

    return {
        "id": file_id,
        "metadata": metadata,
        "messages":messages
    }


def threading_processor(service_account_path, file_id, results):
    results.append(download_and_parse_notebook(service_account_path, file_id))


threads = []
parsed_conversations = []
for i in range(tasks_df.shape[0]):
    file_id = tasks_df["task_link"][i].split("/")[-1]
    thread = threading.Thread(target=threading_processor, args=(service_account_path, file_id, parsed_conversations))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()


Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


Exception in thread Thread-3952:
Traceback (most recent call last):
  File "/home/joe96/miniconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/joe96/miniconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_521/2184826214.py", line 115, in threading_processor
  File "/tmp/ipykernel_521/2184826214.py", line 83, in download_and_parse_notebook
  File "/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/googleapiclient/_helpers.py", line 130, in positional_wrapper
    return wrapped(*args, **kwargs)
  File "/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/googleapiclient/http.py", line 780, in next_chunk
    raise HttpError(resp, content, uri=self._uri)
googleapiclient.errors.HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/1rfNQU__74pEdovonm_-u6yrhF0UsAa2C?alt=media 

Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


Exception in thread Thread-3950:
Traceback (most recent call last):
  File "/home/joe96/miniconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/joe96/miniconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_521/2184826214.py", line 115, in threading_processor
  File "/tmp/ipykernel_521/2184826214.py", line 83, in download_and_parse_notebook
  File "/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/googleapiclient/_helpers.py", line 130, in positional_wrapper
    return wrapped(*args, **kwargs)
  File "/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/googleapiclient/http.py", line 780, in next_chunk
    raise HttpError(resp, content, uri=self._uri)
googleapiclient.errors.HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/1lHYB-8JiU67LlaqjvaRuLYUbetxWbnD5%23scrollTo

Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


In [17]:
from datetime import datetime

def get_number_of_turns(messages):

    initial_role = messages[0]["role"]

    count = 0
    for message in messages:
        if message["role"] == initial_role:
            count += 1
    return count


def get_closest_email_match(query, choices):
    """
    Get the closest match(es) to a query string from a list of choices.

    :param query: The query string.
    :param choices: A list of strings to match against.

    :return: The best match and the score.
    """
    best_email = None
    best_score = 0
    for choice in choices:
        score = fuzz.ratio(query, choice)
        if score > best_score and score > 25:
            best_score = score
            best_email = choice

    return best_email, best_score


def standardize_date_format(date):
    """
    Given a date string, standardize the date format to YYYY/MM/DD.
    """
    if date is None:
        return ""
    try:
        # Parse the date string into a datetime object
        standardized_date = datetime.strptime(date, "%Y/%m/%d")
    except ValueError:
        try:
            # Attempt to parse other common formats here
            # Example: MM/DD/YYYY
            standardized_date = datetime.strptime(date, "%m/%d/%Y")
        except ValueError:
            return "Invalid date format"

    # Format the datetime object into the desired string format
    return standardized_date.strftime("%Y/%m/%d")

not_found_emails = set()
metadata_only = []
for conversation in parsed_conversations:
    try:
        # Extract actual number of turns
        conversation["metadata"]["actual_turns"] = get_number_of_turns(conversation["messages"])

        try:
            tracking_record = tasks_df[tasks_df["task_link"].str.contains(conversation["id"])].iloc[0].to_dict()
        except IndexError:
            print("IndexError for id:", conversation["id"])
            continue

        # Get Author email
        conversation["metadata"]["assigned_to_email"] = tracking_record["assigned_to_email"]

        # Get duration
        conversation["metadata"]["duration_mins"] = tracking_record["duration_mins"]

        # Get Completion Date
        conversation["metadata"]["completion_date"] = standardize_date_format(tracking_record["completion_date"])

        try:
            contrib_entry = contributors_df[contributors_df["Email"] == tracking_record["assigned_to_email"]].iloc[0]
        except IndexError:
            email, _ = get_closest_email_match(tracking_record["assigned_to_email"], contributors_df["Email"].tolist())

            if email is not None:
                contrib_entry = contributors_df[contributors_df["Email"] == email].iloc[0]
                conversation["metadata"]["assigned_to_email"] = email
            else:
                not_found_emails.add(tracking_record["assigned_to_email"])
                print("IndexError for email:", tracking_record["assigned_to_email"])
                continue

        # Get Join Date
        conversation["metadata"]["joined_on"] = contrib_entry["Joined on"]

        # Get Team
        try:
            conversation["metadata"]["team"] = contrib_entry["Source"]
        except IndexError:
            print("IndexError for email:", tracking_record["assigned_to_email"])
            conversation["metadata"]["team"] = "Unknown"

        metadata_only.append(conversation["metadata"])
    except:
        print("Error for id:", conversation["id"])
        continue

metadata_only_df = pd.DataFrame(metadata_only)

IndexError for email: 
Error for id: 1eADOs9fvaqsZCFHeip02H0SpwxcEQPUW
IndexError for email: 


## Filter data on full timers who joined on 22/12/2023

In [6]:
# metadata_only_df = metadata_only_df[metadata_only_df["team"]=="Vetting"]

start_date = '2024-01-01'
end_date = '2024-01-19'
metadata_only_df["completion_date"] = metadata_only_df["completion_date"].replace("Invalid date format", "")
metadata_only_df["completion_date"] = pd.to_datetime(metadata_only_df["completion_date"], errors='coerce')
metadata_only_df["completion_date_str"] = metadata_only_df["completion_date"].dt.strftime('%Y-%m-%d')
metadata_only_df = metadata_only_df[(metadata_only_df['completion_date'] >= start_date) & (metadata_only_df['completion_date'] <= end_date)]
metadata_only_df = metadata_only_df[metadata_only_df["team"]=="Vetting"]

# metadata_only_df = metadata_only_df[metadata_only_df["joined_on"]=="12/28/2023"]
metadata_only_df

,topic,type,target_turns,actual_turns,assigned_to_email,duration_mins,completion_date,joined_on,team,completion_date_str
342,unit_testing_methodology > integration_testing,modification,2+,4,gedeon.a@turing.com,44,2024-01-02,12/22/2023,Vetting,2024-01-02
344,web_development > web_scraping,query,6,6,gedeon.a@turing.com,58,2024-01-01,12/22/2023,Vetting,2024-01-01
348,web_development > web_development_tools,query,2,2,martinho.h@turing.com,50,2024-01-01,12/28/2023,Vetting,2024-01-01
349,algorithms > by_data_structure > graphs,query,1,1,aarunik.g@turing.com,40,2024-01-01,12/28/2023,Vetting,2024-01-01
353,web_development > web_development_interview_qu...,query,2,2,khalid.s@turing.com,45,2024-01-02,12/28/2023,Vetting,2024-01-02
...,...,...,...,...,...,...,...,...,...,...
3611,NaN,NaN,1-4,3,abdullah.i@turing.com,45,2024-01-05,12/22/2023,Vetting,2024-01-05
3614,NaN,query,NaN,2,andranik.g@turing.com,30,2024-01-12,12/28/2023,Vetting,2024-01-12
3615,unit_testing_methodology > code_coverage,query,1-5,2,singh.r@turing.com,30,2024-01-04,12/28/2023,Vetting,2024-01-04
3616,algorithms > by_topic > geometry,modification,3-7,4,singh.r@turing.com,40,2024-01-04,12/28/2023,Vetting,2024-01-04


In [7]:
# Process the duration_mins column to make into int

def process_duration_mins(duration_mins):
    if duration_mins == "" or duration_mins is None:
        return 15
    elif isinstance(duration_mins, int):
        return duration_mins
    else:
        return int(duration_mins.split(" ")[0])
    
metadata_only_df["duration_mins"] = metadata_only_df["duration_mins"].apply(process_duration_mins)

trainer_avg_turn_duration = metadata_only_df.groupby("assigned_to_email").agg({"duration_mins": "mean", "actual_turns": "mean"}).reset_index()
trainer_avg_turn_duration["avg_turn_duration"] = trainer_avg_turn_duration["duration_mins"] / trainer_avg_turn_duration["actual_turns"]
trainer_avg_turn_duration = trainer_avg_turn_duration.sort_values("avg_turn_duration", ascending=True)
trainer_avg_turn_duration = trainer_avg_turn_duration.rename(columns={
    "duration_mins": "avg_mins_per_convo",
    "actual_turns": "avg_turns_per_convo",
    "avg_turn_duration": "avg_mins_per_turn"
})
trainer_avg_turn_duration

,assigned_to_email,avg_mins_per_convo,avg_turns_per_convo,avg_mins_per_turn
30,sudharchith.s@turing.com,36.454545,4.672727,7.801556
11,elsadek.a@turing.com,37.729412,4.694118,8.037594
7,armas.j@turing.com,47.282609,5.347826,8.841463
19,kumbar.r@turing.com,78.223684,8.381579,9.332810
5,andranik.g@turing.com,40.300000,3.930000,10.254453
6,archit.k@turing.com,48.454545,4.381818,11.058091
0,aarunik.g@turing.com,56.923077,5.131868,11.092077
32,zubair.m@turing.com,52.784314,4.705882,11.216667
17,khalid.s@turing.com,45.376344,3.935484,11.530055
1,abdul.r@turing.com,43.418919,3.716216,11.683636


In [8]:
reviews = reviews_df[reviews_df["Author Email"].isin(trainer_avg_turn_duration["assigned_to_email"].tolist())]
reviews = reviews.astype({"Code Quality": "int32", "Language Quality": "int32"})
reviews["avg_quality_score"] = (reviews["Code Quality"] + reviews["Language Quality"]) / 2

trainer_avg_quality = reviews.groupby("Author Email").agg({"avg_quality_score": "mean", "Timestamp":"count"}).reset_index()
trainer_avg_quality = trainer_avg_quality.rename(columns={"Timestamp": "total_reviews"})
trainer_avg_quality = trainer_avg_quality.sort_values("avg_quality_score", ascending=False)
trainer_avg_quality

,Author Email,avg_quality_score,total_reviews
7,armas.j@turing.com,4.954545,11
31,zain.v@turing.com,4.833333,9
18,kishore.g@turing.com,4.833333,3
12,freitas.g@turing.com,4.812500,8
27,safi.u@turing.com,4.788462,26
21,marcus.a@turing.com,4.785714,7
14,ishwar.b@turing.com,4.777778,9
29,singh.r@turing.com,4.727273,11
22,martinho.h@turing.com,4.666667,6
11,elsadek.a@turing.com,4.657895,19


In [9]:
trainer_throughput = metadata_only_df.groupby("assigned_to_email").agg({"topic": "count", "duration_mins":"sum", "actual_turns":"sum"}).reset_index()
trainer_throughput = trainer_throughput.sort_values("topic", ascending=False)
trainer_throughput = trainer_throughput.rename(columns={
    "topic": "total_convos",
    "duration_mins": "total_mins",
    "actual_turns": "total_turns"
})
trainer_throughput.sort_values("total_turns", ascending=False)

,assigned_to_email,total_convos,total_mins,total_turns
19,kumbar.r@turing.com,8,5945,637
7,armas.j@turing.com,15,4350,492
0,aarunik.g@turing.com,34,5180,467
8,caram.v@turing.com,39,4925,421
11,elsadek.a@turing.com,10,3207,399
27,safi.u@turing.com,21,5020,397
5,andranik.g@turing.com,27,4030,393
28,santiago.c@turing.com,22,4739,392
13,gedeon.a@turing.com,28,7143,373
17,khalid.s@turing.com,27,4220,366


In [10]:
import numpy as np
import pandas as pd


def extract_top_level_topic(topic):
    if isinstance(topic, str):
        return topic.split(" > ")[0]
    else:
        return topic


def calculate_diversity(df, group_by="assigned_to_email", count_column="count", entropy_column="diversity"):
    """
    Calculate the diversity of a dataframe.

    :param df: The dataframe to calculate diversity on.
    :param group_by: The column to group by.
    :param count_column: The column to count.
    :param entropy_column: The column to store the entropy in.
    """
    # Step 1: Normalize counts
    total_counts = df.groupby(group_by)[count_column].transform('sum')
    df['normalized_count'] = df[count_column] / total_counts

    # Step 2: Calculate entropy
    df['entropy_component'] = -df['normalized_count'] * np.log2(df['normalized_count'])
    entropy = df.groupby(group_by)['entropy_component'].sum().reset_index()

    entropy.columns = [group_by, entropy_column]
    return entropy


metadata_only_df["top_level_topic"] = metadata_only_df["topic"].apply(extract_top_level_topic)

# Aggregation of Count of tasks per trainer per top level topic in a new variable trainer_diversity
trainer_diversity = metadata_only_df.groupby(["assigned_to_email", "top_level_topic"]).agg({"topic": "count"}).reset_index()
trainer_diversity = trainer_diversity.rename(columns={"topic": "count"})
trainer_diversity = trainer_diversity.sort_values("count", ascending=False)

# Calculate entropy
topleveltopic_diversity = calculate_diversity(trainer_diversity, entropy_column="topic_diversity")
topleveltopic_diversity

,assigned_to_email,topic_diversity
0,aarunik.g@turing.com,1.298747
1,abdul.r@turing.com,1.740653
2,abdullah.i@turing.com,1.418026
3,adil.m@turing.com,1.491115
4,aman.s@turing.com,1.240671
5,andranik.g@turing.com,0.918296
6,archit.k@turing.com,0.918296
7,armas.j@turing.com,0.970951
8,caram.v@turing.com,1.913198
9,cedric.l@turing.com,1.485475


In [11]:
# Hour Tracking Utilization



In [12]:
dpi_df = trainer_avg_turn_duration.merge(trainer_avg_quality, left_on="assigned_to_email", right_on="Author Email", how="inner")
dpi_df = dpi_df.merge(trainer_throughput, on="assigned_to_email", how="inner")
dpi_df = dpi_df.merge(topleveltopic_diversity, on="assigned_to_email", how="inner")
dpi_df = dpi_df.drop(columns=["Author Email"])
dpi_df

,assigned_to_email,avg_mins_per_convo,avg_turns_per_convo,avg_mins_per_turn,avg_quality_score,total_reviews,total_convos,total_mins,total_turns,topic_diversity
0,sudharchith.s@turing.com,36.454545,4.672727,7.801556,3.906250,16,8,2005,257,1.405639
1,elsadek.a@turing.com,37.729412,4.694118,8.037594,4.657895,19,10,3207,399,0.721928
2,armas.j@turing.com,47.282609,5.347826,8.841463,4.954545,11,15,4350,492,0.970951
3,kumbar.r@turing.com,78.223684,8.381579,9.332810,4.318182,11,8,5945,637,0.000000
4,andranik.g@turing.com,40.300000,3.930000,10.254453,4.281250,16,27,4030,393,0.918296
5,archit.k@turing.com,48.454545,4.381818,11.058091,4.545455,22,3,2665,241,0.918296
6,aarunik.g@turing.com,56.923077,5.131868,11.092077,4.321429,14,34,5180,467,1.298747
7,zubair.m@turing.com,52.784314,4.705882,11.216667,4.125000,12,8,2692,240,1.811278
8,khalid.s@turing.com,45.376344,3.935484,11.530055,4.500000,13,27,4220,366,2.253077
9,abdul.r@turing.com,43.418919,3.716216,11.683636,4.653846,13,26,3213,275,1.740653


In [13]:
def transform_to_zscore(sequence):
    """
    Calculate the z-score of a column.

    :param df: The dataframe to calculate z-score on.
    :param column: The column to calculate z-score on.
    """
    return (sequence - sequence.mean()) / sequence.std(ddof=0)


dpi_normalized_df = dpi_df.copy()

# Loop on all numerical columns and transform to z-score
for column in dpi_normalized_df.columns:
    if column != "assigned_to_email":
        dpi_normalized_df[column] = transform_to_zscore(dpi_normalized_df[column])

dpi_normalized_df

,assigned_to_email,avg_mins_per_convo,avg_turns_per_convo,avg_mins_per_turn,avg_quality_score,total_reviews,total_convos,total_mins,total_turns,topic_diversity
0,sudharchith.s@turing.com,-1.291218,0.722917,-1.170536,-1.333620,0.729637,-0.716340,-0.783639,0.045042,0.157575
1,elsadek.a@turing.com,-1.204316,0.741466,-1.135620,0.544646,1.285835,-0.509768,-0.058571,0.962978,-0.923752
2,armas.j@turing.com,-0.553114,1.308358,-1.016707,1.285940,-0.197361,0.006664,0.630908,1.564162,-0.529909
3,kumbar.r@turing.com,1.556011,3.939208,-0.944024,-0.304254,-0.197361,-0.716340,1.593042,2.501492,-2.065521
4,andranik.g@turing.com,-1.029089,0.078829,-0.807689,-0.396542,0.729637,1.246099,0.437878,0.924192,-0.613185
5,archit.k@turing.com,-0.473228,0.470642,-0.688810,0.263672,1.842034,-1.232772,-0.385515,-0.058388,-0.613185
6,aarunik.g@turing.com,0.104037,1.121081,-0.683783,-0.296141,0.358838,1.969103,1.131580,1.402554,-0.011480
7,zubair.m@turing.com,-0.178085,0.751669,-0.665353,-0.786991,-0.011961,-0.716340,-0.369228,-0.064852,0.799116
8,khalid.s@turing.com,-0.683056,0.083584,-0.618995,0.150087,0.173438,1.246099,0.552490,0.749655,1.497846
9,abdul.r@turing.com,-0.816485,-0.106563,-0.596276,0.534530,0.173438,1.142813,-0.054951,0.161400,0.687419


In [14]:
# Create a final score column as a weighted average of all the columns

weights = {
    # Utilization = 1
    "avg_tracked_mins_per_convo": 0.5,
    "avg_tracked_mins_per_turn": 0.5,

    # Efficiency = 1
    "avg_mins_per_convo": 0.3,
    "avg_mins_per_turn": 0.7  ,

    # Quality = 2
    "avg_quality_score": 2,

    # Throughput = 2
    "total_convos": 0.6,
    "total_turns": 1.4,

    # Diversity = 1
    "topic_diversity": 0.7,
    "turns_diversity": 0.3
}


greater_is_better = {
    # Utilization = 1
    "avg_tracked_mins_per_convo": False,
    "avg_tracked_mins_per_turn": False,

    # Efficiency = 1
    "avg_mins_per_convo": False,
    "avg_mins_per_turn": False,

    # Quality = 2
    "avg_quality_score": True,

    # Throughput = 2
    "total_convos": True,
    "total_turns": True,

    # Diversity = 1
    "topic_diversity": True,
    "turns_diversity": True
}


dpi_normalized_df["final_score"] = 0
count_cols = 0
for column in weights.keys():
    try:
        if greater_is_better[column]:
            dpi_normalized_df["final_score"] += dpi_normalized_df[column] * weights[column]
        else:
            dpi_normalized_df["final_score"] += (dpi_normalized_df[column]*(-1)) * weights[column]
            dpi_normalized_df[column] = dpi_normalized_df[column]*(-1)
        count_cols += 1
    except KeyError:
        print("KeyError for column:", column)
        continue


dpi_normalized_df["final_score"] = dpi_normalized_df["final_score"] / count_cols
dpi_normalized_df = dpi_normalized_df.sort_values("final_score", ascending=False)
dpi_normalized_df

KeyError for column: avg_tracked_mins_per_convo
KeyError for column: avg_tracked_mins_per_turn
KeyError for column: turns_diversity


,assigned_to_email,avg_mins_per_convo,avg_turns_per_convo,avg_mins_per_turn,avg_quality_score,total_reviews,total_convos,total_mins,total_turns,topic_diversity,final_score
10,caram.v@turing.com,0.452244,0.285495,0.594101,0.517569,0.915036,2.485535,0.977759,1.105194,0.960308,0.882915
2,armas.j@turing.com,0.553114,1.308358,1.016707,1.285940,-0.197361,0.006664,0.630908,1.564162,-0.529909,0.878733
16,safi.u@turing.com,0.388128,0.079430,0.454090,0.870917,2.583632,0.626381,1.035065,0.950050,1.396881,0.809975
11,singh.r@turing.com,0.870155,-0.252987,0.546844,0.718013,-0.197361,1.349385,0.449943,0.562189,1.240844,0.757525
22,gedeon.a@turing.com,-0.088182,-0.762074,-0.508212,0.462447,-0.753559,1.349385,2.315698,0.794905,2.098042,0.655636
8,khalid.s@turing.com,0.683056,0.083584,0.618995,0.150087,0.173438,1.246099,0.552490,0.749655,1.497846,0.630676
9,abdul.r@turing.com,0.816485,-0.106563,0.596276,0.534530,0.173438,1.142813,-0.054951,0.161400,0.687419,0.520706
6,aarunik.g@turing.com,-0.104037,1.121081,0.683783,-0.296141,0.358838,1.969103,1.131580,1.402554,-0.011480,0.498693
1,elsadek.a@turing.com,1.204316,0.741466,1.135620,0.544646,1.285835,-0.509768,-0.058571,0.962978,-0.923752,0.440201
14,santiago.c@turing.com,0.411194,0.211798,0.536270,0.150087,-0.197361,0.729668,0.865560,0.917728,0.040446,0.424976


In [15]:
# Add final score to the dpi_df
dpi_df = dpi_df.merge(dpi_normalized_df[["assigned_to_email", "final_score"]], on="assigned_to_email", how="inner")
dpi_df = dpi_df.sort_values("final_score", ascending=False)
dpi_df

,assigned_to_email,avg_mins_per_convo,avg_turns_per_convo,avg_mins_per_turn,avg_quality_score,total_reviews,total_convos,total_mins,total_turns,topic_diversity,final_score
10,caram.v@turing.com,48.762376,4.168317,11.698337,4.647059,17,39,4925,421,1.913198,0.882915
2,armas.j@turing.com,47.282609,5.347826,8.841463,4.954545,11,15,4350,492,0.970951,0.878733
16,safi.u@turing.com,49.702970,3.930693,12.644836,4.788462,26,21,5020,397,2.189238,0.809975
11,singh.r@turing.com,42.631579,3.547368,12.017804,4.727273,11,28,4050,337,2.090578,0.757525
22,gedeon.a@turing.com,56.690476,2.960317,19.150134,4.625000,8,28,7143,373,2.632574,0.655636
8,khalid.s@turing.com,45.376344,3.935484,11.530055,4.500000,13,27,4220,366,2.253077,0.630676
9,abdul.r@turing.com,43.418919,3.716216,11.683636,4.653846,13,26,3213,275,1.740653,0.520706
6,aarunik.g@turing.com,56.923077,5.131868,11.092077,4.321429,14,34,5180,467,1.298747,0.498693
1,elsadek.a@turing.com,37.729412,4.694118,8.037594,4.657895,19,10,3207,399,0.721928,0.440201
14,santiago.c@turing.com,49.364583,4.083333,12.089286,4.500000,11,22,4739,392,1.331580,0.424976


In [69]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

load_dotenv(find_dotenv())
opanai = OpenAI()


def fix_missing_roles(messages):
    """
    Fix missing roles in a list of messages.

    :param messages: The list of messages.
    """
    def predict_role(messages_subsequence):
        try:
            response = opanai.chat.completions.create(
                model="gpt-4-1106-preview",
                messages=[
                    {"role":"system", "content": "Your task is to accurately predict whether the empty role is a User or an Assistant. You are only allowed to reply with a single word: 'User' or 'Assistant'."},
                    {"role":"user", "content": f"Here's a part of the conversation including an empty role:\n\n{messages_subsequence}"}
                ],
                temperature=0,
                seed=42
            )
            print(response.choices[0])
            missing_role = response.choices[0].message.content
            assert missing_role in ["User", "Assistant"]
            return missing_role, None
        except Exception as e:
            return None, e

    errors = []
    for i in range(len(messages)):
        if messages[i]["role"] == "":
            subsequence = messages[max(0, i-2):min(len(messages), i+3)]
            messages[i]["role"], error = predict_role(subsequence)
            if error is not None:
                errors.append(error)
    return messages, errors



test = [
    {'role': 'user', 'content': 'Hello'},
    {'role': '', 'content': 'How can I help you?'},
    {'role': 'user', 'content': 'I have a question'},
    {'role': 'assistant', 'content': 'Sure, what is it?'},
    {'role': '', 'content': 'Can you explain this concept to me?'},
    {'role': 'assistant', 'content': 'Of course, here is a brief explanation'},
    {"role": "assistant", "content": "print('Hello World')"},
    {"role": "", "content": "This print should explain the concept for you"},
    {'role': 'user', 'content': 'Thank you!'}
]

fix_missing_roles(test)

Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='User', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='User', role='assistant', function_call=None, tool_calls=None), logprobs=None)


([{'role': 'user', 'content': 'Hello'},
  {'role': 'Assistant', 'content': 'How can I help you?'},
  {'role': 'user', 'content': 'I have a question'},
  {'role': 'assistant', 'content': 'Sure, what is it?'},
  {'role': 'User', 'content': 'Can you explain this concept to me?'},
  {'role': 'assistant', 'content': 'Of course, here is a brief explanation'},
  {'role': 'assistant', 'content': "print('Hello World')"},
  {'role': 'User', 'content': 'This print should explain the concept for you'},
  {'role': 'user', 'content': 'Thank you!'}],
 [])